In [1]:
import matplotlib.pyplot as plt
import pyvista
import ufl
import numpy as np
import dolfinx
from petsc4py import PETSc
from mpi4py import MPI
from dolfinx import fem, mesh, io, plot
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc
import time

# Define temporal parameters
# Time  ----------------------------------------------------------------------
t0=time.time()
t = 0  # Start time
Total_time = 5  # Final time
num_steps = 50
dt = Total_time / num_steps  # time step size

# Boundary condtions --------------------------------------------------------- 
# initial temperature (K), and brake pad temperature
Ti = 250
Tm = 250

# This step is for the initial process. Variable need to be projected form Q onto V
def project(function, space):
    p = TrialFunction(space)
    q = TestFunction(space)
    a = inner(p, q) * dx
    L = inner(function, q) * dx
    problem = LinearProblem(a, L, bcs = [])
    return problem.solve()

# PhysicalProperties
# m to mm
unit_m_to_mm = 1e3
t_u = unit_m_to_mm

# density (kg.m^-3), capacity (J/Kg.K), conductivity (W/m.K)
rho , c ,k = 7850 /(t_u**3), 462 , 48 / t_u
mu , P_brake , v_angualr , r_disc , heat_distribution = 0.35, 246000, 48.3, 0.25, 0.88
h_input = mu * P_brake * v_angualr * r_disc * heat_distribution /1e9 *100

g = h_input

# f is the heat source,unit is w/m3

RhoCp_values = rho *c
Conductivity_values = k
Tinit_values = Ti

from dolfinx.io import gmshio
from dolfinx import plot
from dolfinx import default_scalar_type
from dolfinx.fem import (Constant, dirichletbc, Function, FunctionSpace, 
                         form,  locate_dofs_topological)
from dolfinx.fem.petsc import LinearProblem, assemble_matrix, create_vector, assemble_vector, apply_lifting, set_bc
from dolfinx.io import XDMFFile, gmshio
from dolfinx.mesh import create_unit_square, locate_entities, locate_entities_boundary,meshtags
from ufl import (SpatialCoordinate, TestFunction, TrialFunction,
                 dx, grad, inner, Measure, dot, jump, FacetNormal, avg, FiniteElement, MixedElement)
from find_com_e import find_common_e

filename = "3-10" # mesh
domain, cell_markers, facet_markers = gmshio.read_from_msh(filename+".msh", MPI.COMM_WORLD, 0, gdim=3)

Q = FunctionSpace(domain, ("DG", 0))
Conductivity = Function(Q)
Conductivity.x.array[:] = np.full_like(1, Conductivity_values, dtype=default_scalar_type)
RhoCp = Function(Q)
RhoCp.x.array[:] = np.full_like(1, RhoCp_values, dtype=default_scalar_type)

T_INIT = Function(Q)
T_INIT.name = "u_n"
# here 1 is the volume 1, the brake disc
T_INIT.x.array[:] = np.full_like(1, Tinit_values, dtype=default_scalar_type)

fdim = domain.topology.dim - 1
import sys
# when change a new computer, below path is inconvenient, need to find a bettwe way to use this function.
sys.path.append('/home/yanjun/Documents/FEniCSx/Tutorial/Mesh/')
from rub_co import get_rub_coordinate
x_co, y_co = get_rub_coordinate()

bc_list = []
for i, (x_c, y_c) in enumerate(zip(x_co, y_co), start=1):
    var_name = f"bc_facets{i}"    
    tag= mesh.locate_entities_boundary(
       domain, fdim, lambda x: (x[0]-x_c)**2 + (x[1]-y_c)**2  <= 324 )
    globals()[var_name] = tag
    bc_list.append(globals()[var_name])
    
# rubbing elements
bc_disc = mesh.locate_entities_boundary(
    domain, fdim , lambda x:  np.isclose(x[2], 20) )

bc_all = find_common_e(bc_disc, bc_list)
x = SpatialCoordinate(domain)



boundaries = [(1, lambda x: (x[0]-x_c)**2 + (x[1]-y_c)**2  <= 324)]
print(boundaries)

#for (marker, locator) in boundaries:
    #facets = locate_entities(domain, fdim, locator)
#print(facets)

facet_indices, facet_markers = [], []
fdim = 2
for (marker, locator) in boundaries:
    facets = locate_entities(domain, fdim, locator)
    facet_indices.append(facets)
    facet_markers.append(np.full_like(facets, marker))
facet_indices = np.hstack(facet_indices).astype(np.int32)
facet_markers = np.hstack(facet_markers).astype(np.int32)
sorted_facets = np.argsort(facet_indices)
facet_tag = meshtags(domain, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])

print(facet_tag)
#print(facet_indices)
ds = Measure('ds', domain=domain, subdomain_data=facet_tag)

/home/yanjun/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


Info    : Reading '3-10.msh'...
Info    : 168 entities
Info    : 10751 nodes
Info    : 43604 elements
Info    : Done reading '3-10.msh'
[(1, <function <lambda> at 0x722e04dd5ab0>)]


In [4]:
xdmf = io.XDMFFile(domain.comm, "temperature4.xdmf", "w")
xdmf.write_mesh(domain)
V = fem.FunctionSpace(domain, ('CG', 1))
uh = fem.Function(V)
uh.name = "uh"
uh = project(T_INIT,V)
xdmf.write_function(uh, t)


# Define solution variable, and interpolate initial solution for visualization in Paraview

#uh.interpolate(initial_condition)
#uh.interpolate(Tinit)

uh = fem.Function(V)
uh.name = "uh"
uh = project(T_INIT,V)
xdmf.write_function(uh, t)

# assembly matrix and vector
bilinear_form = fem.form(a)
linear_form = fem.form(L)

HDF5-DIAG: Error detected in HDF5 (1.10.7) MPI-process 0:
  #000: ../../../src/H5D.c line 152 in H5Dcreate2(): unable to create dataset
    major: Dataset
    minor: Unable to initialize object
  #001: ../../../src/H5Dint.c line 338 in H5D__create_named(): unable to create and link to dataset
    major: Dataset
    minor: Unable to initialize object
  #002: ../../../src/H5L.c line 1605 in H5L_link_object(): unable to create new link to object
    major: Links
    minor: Unable to initialize object
  #003: ../../../src/H5L.c line 1846 in H5L__create_real(): can't insert link
    major: Links
    minor: Unable to insert object
  #004: ../../../src/H5Gtraverse.c line 848 in H5G_traverse(): internal path traversal failed
    major: Symbol table
    minor: Object not found
  #005: ../../../src/H5Gtraverse.c line 624 in H5G__traverse_real(): traversal operator failed
    major: Symbol table
    minor: Callback failed
  #006: ../../../src/H5L.c line 1641 in H5L__link_cb(): name already exists

RuntimeError: Failed to create HDF5 global dataset.